In [1]:
import os, re, glob
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.nasnet import NASNetMobile
from sklearn.model_selection import train_test_split
from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator

Using TensorFlow backend.


In [2]:
SEED = 0
BATCH_SIZE = 64
MODEL_NAME = 'nasnetmobile'
FILE_PATH = '/home/matt/repos/ohia.ai/data'
IMAGE_DIR = 'resized_images'

In [3]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]
full_label_ids = FastLabelEncoder().fit_transform(full_label_list)

In [4]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, full_label_ids, test_size=0.1, random_state=SEED)

In [5]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(train_files, train_ids, n_classes, BATCH_SIZE, augment=True)
valid_generator = PlantNetGenerator(valid_files, valid_ids, n_classes, BATCH_SIZE, augment=False)

In [ ]:
# load pretrained ImageNet model
base_model = NASNetMobile(
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=False
)

# set freeze all layers
for layer in base_model.layers:
    layer.trainable = False
    
# map ImageNet features to 39 plants
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)
    
# compile the model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
)

# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{FILE_PATH}/weights/{MODEL_NAME}.h5',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    ),
]

# train model
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=20,
    epochs=100,    
)

Epoch 1/100
164/165 [============================>.] - ETA: 0s - loss: 2.2042 - top_1_accuracy: 0.3825 - top_3_accuracy: 0.6088 - top_5_accuracy: 0.7092

165/165 [==============================] - 52s 314ms/step - loss: 2.2030 - top_1_accuracy: 0.3826 - top_3_accuracy: 0.6091 - top_5_accuracy: 0.7097 - val_loss: 1.8560 - val_top_1_accuracy: 0.4688 - val_top_3_accuracy: 0.7066 - val_top_5_accuracy: 0.8194
Epoch 2/100
165/165 [==============================] - 42s 254ms/step - loss: 1.5479 - top_1_accuracy: 0.5534 - top_3_accuracy: 0.7814 - top_5_accuracy: 0.8651 - val_loss: 1.5960 - val_top_1_accuracy: 0.5148 - val_top_3_accuracy: 0.7578 - val_top_5_accuracy: 0.8568
Epoch 3/100
165/165 [==============================] - 43s 263ms/step - loss: 1.3547 - top_1_accuracy: 0.5963 - top_3_accuracy: 0.8206 - top_5_accuracy: 0.8951 - val_loss: 1.6034 - val_top_1_accuracy: 0.5000 - val_top_3_accuracy: 0.7587 - val_top_5_accuracy: 0.8620
Epoch 4/100
165/165 [==============================] - 42s 2

In [ ]:
# save model
model.load_weights(f'{FILE_PATH}/weights/{MODEL_NAME}.h5')
tfjs.converters.save_keras_model(model, f'{FILE_PATH}/models/{MODEL_NAME}')